In [18]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [19]:
# image = PIL.Image.open('train_gen/burn through/frame_00031.png')
batch_size = 32
# img_height = 487
# img_width = 400
img_height = 100
img_width = 100

In [20]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'weld_dataset',
#     rescale=1.0/255.0
  validation_split=0.2,
  subset="training",
  seed=0,
  image_size=(img_height, img_width),
#     target_size = (100,100),
    shuffle = False,
  batch_size=batch_size)

Found 8268 files belonging to 6 classes.
Using 6615 files for training.


In [21]:
class_names = train_ds.class_names
print(class_names)

['burn through', 'contamination', 'good weld', 'lack of fusion', 'lack of penetration', 'misalignment']


In [22]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#       for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")

In [23]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(500).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [24]:
normalization_layer = layers.Rescaling(1./255)

In [25]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(normalized_ds))
# first_image = image_batch[0]
# # Notice the pixel values are now in `[0,1]`.
# print(np.min(first_image), np.max(first_image))

In [31]:
from tensorflow.keras.applications import Xception, VGG19, ResNet152, InceptionV3, EfficientNetB0
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input 
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras import Model 
from tensorflow.keras.optimizers import Adam

In [41]:
num_classes = len(class_names)

# model = Sequential([
#   layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
#   layers.Conv2D(16, 3, padding='same', activation='relu'),
#   layers.MaxPooling2D(),
#   layers.Conv2D(32, 3, padding='same', activation='relu'),
#   layers.MaxPooling2D(),
#   layers.Conv2D(64, 3, padding='same', activation='relu'),
#   layers.MaxPooling2D(),
#   layers.Flatten(),
#   layers.Dense(128, activation='relu'),
#   layers.Dense(num_classes, activation='softmax')
# ])

headmodel = VGG19(weights = "imagenet", include_top = False,
             input_tensor = Input(shape = (img_height, img_width, 3)))
model = headmodel.output
model = AveragePooling2D(pool_size= (3, 3))(model)
model = Flatten(name = 'flatten')(model)
model = Dense(512, activation = 'relu')(model)
model = Dropout(0.2)(model)
model = Dense(num_classes, activation = 'softmax')(model)

for layer in headmodel.layers:
    layer.trainable = False
    
final_model = Model(inputs = headmodel.input, outputs = model)



In [42]:
final_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [43]:
final_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0   

In [44]:
epochs=100
history = final_model.fit(
  normalized_ds,
#   validation_data=val_ds,
  epochs=epochs,verbose = 2,
    shuffle = False,
    use_multiprocessing = True 
)

Epoch 1/100


C:\Users\suman\Desktop\venv\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


207/207 - 12s - loss: 1.2917 - accuracy: 0.4890 - 12s/epoch - 58ms/step
Epoch 2/100
207/207 - 10s - loss: 0.7095 - accuracy: 0.7327 - 10s/epoch - 47ms/step
Epoch 3/100
207/207 - 10s - loss: 0.5095 - accuracy: 0.8212 - 10s/epoch - 47ms/step
Epoch 4/100
207/207 - 10s - loss: 0.4116 - accuracy: 0.8621 - 10s/epoch - 48ms/step
Epoch 5/100
207/207 - 10s - loss: 0.4207 - accuracy: 0.8366 - 10s/epoch - 49ms/step
Epoch 6/100
207/207 - 10s - loss: 0.3082 - accuracy: 0.8886 - 10s/epoch - 48ms/step
Epoch 7/100
207/207 - 10s - loss: 0.2927 - accuracy: 0.9005 - 10s/epoch - 48ms/step
Epoch 8/100
207/207 - 10s - loss: 0.2253 - accuracy: 0.9075 - 10s/epoch - 48ms/step
Epoch 9/100
207/207 - 10s - loss: 0.2642 - accuracy: 0.9005 - 10s/epoch - 48ms/step
Epoch 10/100
207/207 - 10s - loss: 0.2401 - accuracy: 0.8912 - 10s/epoch - 48ms/step
Epoch 11/100
207/207 - 10s - loss: 0.1963 - accuracy: 0.9231 - 10s/epoch - 48ms/step
Epoch 12/100
207/207 - 10s - loss: 0.1971 - accuracy: 0.9237 - 10s/epoch - 48ms/step
E

Epoch 98/100
207/207 - 10s - loss: 0.0317 - accuracy: 0.9878 - 10s/epoch - 49ms/step
Epoch 99/100
207/207 - 10s - loss: 0.0344 - accuracy: 0.9890 - 10s/epoch - 49ms/step
Epoch 100/100
207/207 - 10s - loss: 0.0808 - accuracy: 0.9714 - 10s/epoch - 49ms/step


In [46]:
final_model.save('weldModel')

INFO:tensorflow:Assets written to: weldModel\assets
